# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [45]:
problem = '''We are delighted to welcome you, our newest intern on the Analytics team of Massachusetts General Hospital! You have been placed in a challenging role where you will be tasked with solving a real-world problem in the field of medical physics. We are building a pilot program in Boston, and if successful, your work could be applied widely in hospitals with limited capacity in many countries.

You are responsible for determining the best treatment plan for 17 patients who require radiotherapy. Your goal is to optimize the use of two possible treatments: photon therapy and proton therapy. While proton therapy is known to target tumors more precisely, it is also more expensive and has limited capacity in many countries. Therefore, you will need to balance the benefits of proton therapy with its limitations and cost to create an effective treatment plan for each patient.

To determine the best course of action for each patient, you will use a scoring system called the Biological Equivalent Dose (BED). This system allows you to calculate the effectiveness of each patient’s treatment plan by considering the number of proton fractions that can be used while still achieving the highest possible BED.

We have n=17 patients who need radiotherapy. Each patient i needs 15 fractions, which can be photon fractions, proton fractions, or a mix of photon and proton fractions (e.g. 4 proton fractions and 11 photon fractions). We want to use the limited proton therapy capacity as best as possible. We can calculate the BED score for each patient when p proton fractions and 15-p photon fractions are used, as BEDi(p,15-p), i.e., the BED when p proton and 15-p photon fractions are delivered for patient i. The higher the score, the better. 

The data file "ProblemData.csv" contains a 2D matrix of BED scores. It does not have an index. It was made in Excel and saved as csv. The columns are the number of proton fractions and each row represents a patient. In particular, the number at the (i,j) position is the score for patient i receiving j proton fractions. 

Suppose that the total maximal capacity C is 100 proton fractions. To maximize the total BED scores for all the patients, which patients should get proton fractions, and how many should they get? Formulate an integer linear optimization model to solve this problem. Assume you know the value BEDi(j,15-j) for each patient i. '''

## 2. Ask for parameters

In [93]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [94]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

*  Let  **x<sub>ij</sub>** be a binary variable equal to 1 if patient *i* receives *j* proton fractions, and 0 otherwise, for *i* = 1, ..., 17 and *j* = 0, ..., 15. 


# 2. Ask for objective

In [95]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [96]:
Markdown(response2.text)

**Objective function:**

Maximize $\sum_{i=1}^{17} \sum_{j=0}^{15}  BED_{i}(j, 15-j) \cdot x_{ij}$ 


# 3. Ask for constraints

In [97]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [98]:
Markdown(response3.text)

## Constraints:

1. **One treatment plan per patient:** Each patient can have only one treatment plan (i.e., one specific value of *j* proton fractions).
   
   $\sum_{j=0}^{15} x_{ij} = 1,  \forall i \in \{1,...,17\}$

2. **Proton capacity constraint:** The total number of proton fractions assigned to all patients cannot exceed the maximum capacity *C*.

   $\sum_{i=1}^{17} \sum_{j=0}^{15} j \cdot x_{ij} \leq 100$ 

3. **Binary variable constraint:** 

   $x_{ij} \in \{0,1\}, \forall i \in \{1,...,17\}, \forall j \in \{0,...,15\}$ 


## 3. Generate the pyomo code

In [104]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [105]:
#Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample data
BED = {}  # Replace with actual BED values
for i in range(1, 18):
    for j in range(0, 16):
        # Replace with a function to calculate or look up BED values
        BED[i, j] = i * j  

C = 100  # Maximum proton capacity

# Model
model = pyo.ConcreteModel()

# Sets
model.patients = pyo.RangeSet(1, 17)
model.fractions = pyo.RangeSet(0, 15)

# Variables
model.x = pyo.Var(model.patients, model.fractions, within=pyo.Binary)

# Objective function
def objective_rule(model):
    return sum(BED[i, j] * model.x[i, j] for i in model.patients for j in model.fractions)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def one_treatment_plan_rule(model, i):
    return sum(model.x[i, j] for j in model.fractions) == 1
model.one_treatment_plan = pyo.Constraint(model.patients, rule=one_treatment_plan_rule)

def proton_capacity_rule(model):
    return sum(j * model.x[i, j] for i in model.patients for j in model.fractions) <= C
model.proton_capacity = pyo.Constraint(rule=proton_capacity_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
solver.solve(model)

# Print the solution
print("Optimal Solution:")
for i in model.patients:
    for j in model.fractions:
        if pyo.value(model.x[i, j]) > 0.5:
            print(f"Patient {i} receives {j} proton fractions")
```

**Explanation:**

1. **Sample data:** 
   -  The `BED` dictionary is defined to store the BED values. You will need to replace the placeholder comment with actual data or a function to calculate/retrieve BED values based on *i* and *j*.
   -  The `C` variable represents the maximum proton capacity and is set to 100.

2. **Model:** 
   - A concrete model is created.
   - Sets for patients and fractions are defined for clarity and ease of use.

3. **Variables:**
   - The binary variable `x[i, j]` is defined, representing whether patient *i* receives *j* fractions.

4. **Objective function:** 
   - The objective is to maximize the total BED, which is calculated by summing over all patients and fractions.

5. **Constraints:** 
   - `one_treatment_plan`: Ensures each patient receives exactly one treatment plan (one specific value of *j*).
   - `proton_capacity`: Limits the total number of proton fractions assigned to all patients to be less than or equal to the capacity *C*.

6. **Solver:** 
   - A solver ('glpk' in this example) is selected to solve the model. You can choose a different solver based on availability and performance.

7. **Solution:** 
   - The code prints the optimal solution, indicating the number of proton fractions assigned to each patient.

Remember that this code is a starting point and you need to provide the real BED data for it to be functional.


## 4. Input problem data and try running the generated code

In [108]:
import pyomo.environ as pyo

# Sample data
# BED = {}  # Replace with actual BED values
# for i in range(1, 18):
#     for j in range(0, 16):
#         # Replace with a function to calculate or look up BED values
#         BED[i, j] = i * j  
#Data input done by human
df = pd.read_csv('ProblemData.csv', header=None)
BED = {}
for i, row in df.iterrows():
    for j, dose in enumerate(row):
        BED[(i+1, j)] = dose
#End


C = 100  # Maximum proton capacity

# Model
model = pyo.ConcreteModel()

# Sets
model.patients = pyo.RangeSet(1, 17)
model.fractions = pyo.RangeSet(0, 15)

# Variables
model.x = pyo.Var(model.patients, model.fractions, within=pyo.Binary)

# Objective function
def objective_rule(model):
    return sum(BED[i, j] * model.x[i, j] for i in model.patients for j in model.fractions)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def one_treatment_plan_rule(model, i):
    return sum(model.x[i, j] for j in model.fractions) == 1
model.one_treatment_plan = pyo.Constraint(model.patients, rule=one_treatment_plan_rule)

def proton_capacity_rule(model):
    return sum(j * model.x[i, j] for i in model.patients for j in model.fractions) <= C
model.proton_capacity = pyo.Constraint(rule=proton_capacity_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
solver.solve(model)

# Print the solution
print("Optimal Solution:")
for i in model.patients:
    for j in model.fractions:
        if pyo.value(model.x[i, j]) > 0.5:
            print(f"Patient {i} receives {j} proton fractions")
            
print(model.obj())

Optimal Solution:
Patient 1 receives 8 proton fractions
Patient 2 receives 8 proton fractions
Patient 3 receives 3 proton fractions
Patient 4 receives 0 proton fractions
Patient 5 receives 5 proton fractions
Patient 6 receives 0 proton fractions
Patient 7 receives 4 proton fractions
Patient 8 receives 15 proton fractions
Patient 9 receives 4 proton fractions
Patient 10 receives 5 proton fractions
Patient 11 receives 6 proton fractions
Patient 12 receives 0 proton fractions
Patient 13 receives 10 proton fractions
Patient 14 receives 0 proton fractions
Patient 15 receives 10 proton fractions
Patient 16 receives 10 proton fractions
Patient 17 receives 12 proton fractions
8.239999999999998


## 5. Correct the code to verify model viability (optional)

In [86]:
BED

{(1, 0): 0,
 (1, 1): 1,
 (1, 2): 2,
 (1, 3): 3,
 (1, 4): 4,
 (1, 5): 5,
 (1, 6): 6,
 (1, 7): 7,
 (1, 8): 8,
 (1, 9): 9,
 (1, 10): 10,
 (1, 11): 11,
 (1, 12): 12,
 (1, 13): 13,
 (1, 14): 14,
 (1, 15): 15,
 (2, 0): 0,
 (2, 1): 2,
 (2, 2): 4,
 (2, 3): 6,
 (2, 4): 8,
 (2, 5): 10,
 (2, 6): 12,
 (2, 7): 14,
 (2, 8): 16,
 (2, 9): 18,
 (2, 10): 20,
 (2, 11): 22,
 (2, 12): 24,
 (2, 13): 26,
 (2, 14): 28,
 (2, 15): 30,
 (3, 0): 0,
 (3, 1): 3,
 (3, 2): 6,
 (3, 3): 9,
 (3, 4): 12,
 (3, 5): 15,
 (3, 6): 18,
 (3, 7): 21,
 (3, 8): 24,
 (3, 9): 27,
 (3, 10): 30,
 (3, 11): 33,
 (3, 12): 36,
 (3, 13): 39,
 (3, 14): 42,
 (3, 15): 45,
 (4, 0): 0,
 (4, 1): 4,
 (4, 2): 8,
 (4, 3): 12,
 (4, 4): 16,
 (4, 5): 20,
 (4, 6): 24,
 (4, 7): 28,
 (4, 8): 32,
 (4, 9): 36,
 (4, 10): 40,
 (4, 11): 44,
 (4, 12): 48,
 (4, 13): 52,
 (4, 14): 56,
 (4, 15): 60,
 (5, 0): 0,
 (5, 1): 5,
 (5, 2): 10,
 (5, 3): 15,
 (5, 4): 20,
 (5, 5): 25,
 (5, 6): 30,
 (5, 7): 35,
 (5, 8): 40,
 (5, 9): 45,
 (5, 10): 50,
 (5, 11): 55,
 (5, 12): 

## 6. Print the responses

In [89]:
response.text

'## Variables:\n\n* **x<sub>ij</sub>:** Binary variable equal to 1 if patient i receives j proton fractions and 0 otherwise, for i = 1,...,17 and j = 0,...,15. \n'

In [90]:
response2.text

'$$\\max \\sum_{i=1}^{17} \\sum_{j=0}^{15} BED_i(j, 15-j) \\cdot x_{ij}$$ \n'

In [91]:
response3.text

'Constraints:\n\n* **Capacity Constraint:** The total number of proton fractions assigned to patients cannot exceed the available capacity:\n  $$\\sum_{i=1}^{17} \\sum_{j=0}^{15} j \\cdot x_{ij} \\le 100$$\n\n* **Treatment Plan Uniqueness:** Each patient can only have one treatment plan (i.e., a specific number of proton fractions):\n  $$\\sum_{j=0}^{15} x_{ij} = 1, \\quad \\forall i = 1, ..., 17$$\n\n* **Binary Variable Constraint:**\n  $$x_{ij} \\in \\{0, 1\\}, \\quad \\forall i = 1, ..., 17, \\quad j = 0, ..., 15$$ \n'

In [92]:
response4.text

'```python\nimport pyomo.environ as pyo\n\n# Sample Data - Replace with actual data if available\nBED = {} \nfor i in range(1, 18):  \n    for j in range(0, 16):\n        # Replace with actual BED calculation based on i and j\n        BED[i, j] = i * j  \n\n# Model\nmodel = pyo.ConcreteModel()\n\n# Sets\nmodel.I = pyo.RangeSet(1, 17) # Set of patients\nmodel.J = pyo.RangeSet(0, 15) # Set of possible proton fractions\n\n# Variables\nmodel.x = pyo.Var(model.I, model.J, within=pyo.Binary)\n\n# Objective function\ndef objective_rule(model):\n    return sum(BED[i, j] * model.x[i, j] for i in model.I for j in model.J)\nmodel.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)\n\n# Constraints\ndef capacity_constraint(model):\n    return sum(j * model.x[i, j] for i in model.I for j in model.J) <= 100\nmodel.capacity = pyo.Constraint(rule=capacity_constraint)\n\ndef treatment_plan_constraint(model, i):\n    return sum(model.x[i, j] for j in model.J) == 1\nmodel.treatment_plan = pyo.Co